<a href="https://colab.research.google.com/github/anilozdemir/Bee-DCD/blob/main/notebooks/03_MLP_with_hidden_layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get Bee-DCD Repo from GitHub
---

In [2]:
%pwd

'/content'

In [3]:
!git clone https://github.com/anilozdemir/Bee-DCD.git

Cloning into 'Bee-DCD'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 45 (delta 9), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (45/45), done.


## Install `btdm` module

In [4]:
%cd Bee-DCD/src

/content/Bee-DCD/src


In [5]:
!python setup.py develop

running develop
running egg_info
creating BTDM.egg-info
writing BTDM.egg-info/PKG-INFO
writing dependency_links to BTDM.egg-info/dependency_links.txt
writing top-level names to BTDM.egg-info/top_level.txt
writing manifest file 'BTDM.egg-info/SOURCES.txt'
writing manifest file 'BTDM.egg-info/SOURCES.txt'
running build_ext
Creating /usr/local/lib/python3.7/dist-packages/BTDM.egg-link (link to .)
Adding BTDM 1.0 to easy-install.pth file

Installed /content/Bee-DCD/src
Processing dependencies for BTDM==1.0
Finished processing dependencies for BTDM==1.0


# Using Perturbed MLPs

In [6]:
import time
import matplotlib.pyplot as P
from matplotlib import style
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm.notebook import trange, tqdm
import itertools

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.distributions import Categorical
import torch.nn.utils as utils

from btdm.networks import *
from btdm.environments import DCD_SingleStep
from btdm.utils import * 
style.use('ggplot')
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from joblib import Parallel, delayed

# Construct Exp Class and MLP

In [7]:
class agent():
    '''
    Custom REINFORCE agent class
    INPUT: env, model and optimiser
    '''
    def __init__(self, env, model, optim):
        self.env   = env
        self.model = model
        self.optim = optim
    
    def run(self, nEpoch=1000, returnDF=False, silent=False):
        self.nEpoch   = nEpoch         
        # reverse assignment
        env       = self.env  
        model     = self.model
        optimizer = self.optim
        total_rewards = 0
        logs = []
        for epoch in trange(nEpoch, disable=silent):
            state   = env.reset()
            probs    = model(Variable(torch.Tensor([state])))[0] # predict 
            action   = torch.multinomial(probs, 1).item()
            log_prob = torch.log(probs[action])
            next_state, reward, done, _    = env.step(action)
            total_rewards += reward
            logs.append([epoch, env.cueID, action, np.round(log_prob.cpu().detach().numpy(),3), reward, total_rewards])                
            # update time, at every epoch
            policy_gradient = [-log_prob * reward]
            loss = torch.stack(policy_gradient).sum()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        # Performance operations
        cols = ['ep', 'cueID', 'action', 'logprob', 'reward', 'totalRewards']
        self.logs     = logs
        self.df       = pd.DataFrame(logs, columns = cols)
        self.perf     = np.around(self.df['reward'].mean(),3)
        self.nParam   = sum([np.prod(param.data.shape) for _, param in model.named_parameters() if param.requires_grad])
        
        if returnDF:
            return self.df

In [8]:
class NoiseLayer(nn.Module):
    def __init__(self, noise=0.1):
        super().__init__()
        self.noise = noise

    def forward(self, x):
        # if noise is nonzero, add Gaussian noise to the incoming tensor, else, do nothing
        return x + torch.normal(0, self.noise, size=x.shape) if self.noise != 0 else x
        
    def extra_repr(self):
        return f'noise_level={self.noise}'

In [9]:
def experiment(nHidden, nHiddenLayer, lR, bias, perturbedNoise, nEpoch=1000, returnDF=False, silent=True):
    env     = DCD_SingleStep()
    nInput  = env.observation_space.shape[0]
    nOutput = env.action_space.n
    hiddenLayers = [[nn.Linear(nHidden, nHidden,bias=bias), NoiseLayer(noise=perturbedNoise), nn.ReLU()] for _ in range(nHiddenLayer)]
    net     = nn.Sequential(nn.Linear(nInput, nHidden,bias=bias), NoiseLayer(noise=perturbedNoise), nn.ReLU(), *list(itertools.chain(*hiddenLayers)), nn.Linear(nHidden, nOutput,bias=bias), softMax(temp=1))
    opt     = optim.Adam(net.parameters(), lr=lR)
    exp     = agent(env, net, opt)
    exp.run(nEpoch=nEpoch, returnDF=returnDF, silent=silent)
    return exp

# Experiments

In [ ]:
nTrial  = 50
nBin    = 50
nEpoch  = 500
bias    = True
hidden  = 20 

Layers  = [1, 5, 10]
Noises  = [0.0, 0.5, 1.0]
lRs     = [0.0005, 0.0003, 0.0002, 0.001, 0.005, 0.003, 0.002, 0.01] #[0.00001, 0.00005, 0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5]


PERF_RAW= np.zeros((len(Noises), len(Layers), len(lRs), nTrial, nEpoch))
PERF    = np.zeros((len(Noises), len(Layers), len(lRs), nTrial, nEpoch-nBin+1))
log     = []

for iN, noise in enumerate(tqdm(Noises, leave=False, desc="noise")):
    for iL, layer in enumerate(tqdm(Layers, leave=False, desc="layer")):
        for ilR, lR in enumerate(tqdm(lRs, leave=False, desc="lRate")):
            # function for parallel running
            def runParallel():
                exp = experiment(nHidden=hidden, nHiddenLayer=layer, lR=lR, bias=bias, perturbedNoise=noise, nEpoch=nEpoch, returnDF=False, silent=True)
                return exp
            # run in parallel
            EXP = np.array(Parallel(n_jobs=min(50,nTrial),verbose=0)(delayed(runParallel)() for _ in range(nTrial)))
            # Go through the results and extract what is needed
            for iT, exp in enumerate(EXP): 
                PERF[iN, iL, ilR, iT] = rolling_sum(np.array(exp.df['reward']), normalise=True, n=nBin)
                PERF_RAW[iN, iL, ilR, iT] = exp.df['reward']
                log.append([noise, layer, lR, iT, exp.nParam, exp.perf])

noise:   0%|          | 0/3 [00:00<?, ?it/s]

layer:   0%|          | 0/3 [00:00<?, ?it/s]

lRate:   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
df = pd.DataFrame(log, columns=['noise','layer', 'lRate', 'trial', 'nParam', 'perf'])

In [ ]:
df

# Select Best lR for each layer size and each noise level

In [ ]:
scores = []
for iN, noise in enumerate(Noises):
    for iL, layer in enumerate(Layers):
        scores.append([noise, layer, *np.mean(PERF[iN,iL],axis=1)[:,-1]])
sDF = pd.DataFrame(scores, columns = ['noise', 'layer'] +['perf-lr-'+str(lr) for lr in lRs])
sDF

## Programatically find the best lR

In [ ]:
optlR = np.zeros((3,3))
for iN in range(3):
    print(f'>> fastest to reach to 1 for noise: {Noises[iN]}')
    for iL, layer in enumerate(Layers):
        perf  = np.mean(PERF[iN,iL],axis=1)[:,-1]
        index = np.argwhere(perf==1)
        if len(index) != 0: # if there is at least one item reached perf=1; if so, get quickest
            quickest = index[np.argmin([np.argwhere(np.mean(PERF[iN,iL],axis=1)[ind]==1)[0,1] for ind in index])][0] # get the lR-index of the quickest lR that reaches 1
            print(f'layer: {layer}\tlR: {lRs[quickest]}')
            optlR[iN,iL] = lRs[quickest]
        else: # if there is no perf=1; get the highest performance
            print(f'layer: {layer}\tlR: {lRs[np.argmax(perf)]}')
            optlR[iN,iL] = lRs[np.argmax(perf)]
    

In [ ]:
optlR

In [ ]:
# insert optimised lR and its arg in the list to sDF
sDF.insert(2, 'optLR'    , optlR.flatten())
sDF.insert(3, 'optLR-arg', list(map(lambda x: lRs.index(x), optlR.flatten())))
sDF

# Perform Post Evaluation with 500 Trials
Using `optLR`

In [ ]:
nTrial  = 500
nBin    = 50
nEpoch  = 1000

PERF1    = np.zeros((len(Noises), len(Layers), nTrial, nEpoch-nBin+1))
PERF_RAW1= np.zeros((len(Noises), len(Layers), nTrial, nEpoch))
log1     = []

for iN, noise in enumerate(tqdm(Noises, leave=False, desc="noise")):
    for iL, layer in enumerate(tqdm(Layers, leave=False, desc="layer")):
        # get optimised learning rate
        lR  = optlR[iN, iL]
        # function for parallel running
        def runParallel():
            exp = experiment(nHidden=hidden, nHiddenLayer=layer, lR=lR, bias=bias, perturbedNoise=noise, nEpoch=nEpoch, returnDF=False, silent=True)
            return exp
        # run in parallel
        EXP = np.array(Parallel(n_jobs=min(50,nTrial),verbose=0)(delayed(runParallel)() for _ in range(nTrial)))
        # Go through the results and extract what is needed
        for iT, exp in enumerate(EXP): 
            PERF1[iN, iL, iT] = rolling_sum(np.array(exp.df['reward']), normalise=True, n=nBin)
            PERF_RAW1[iN, iL, iT] = exp.df['reward']
            log1.append([noise, layer, lR, iT, exp.nParam, exp.perf])

In [ ]:
df1 = pd.DataFrame(log1, columns=['noise','layer', 'lRate', 'trial', 'nParam', 'perf'])

# Plotting & Analysis

In [ ]:
sns.set_context('talk',font_scale=1.2)

nBin = 50
fig, ax = P.subplots(1,len(Noises),figsize=(26,6))
for iN,noise in enumerate(Noises):
    for iL,layer in enumerate(Layers):
        data = np.array([rolling_sum(perf,n=nBin,normalise=True) for perf in PERF_RAW1[iN, iL]])
        mean = np.mean(data,axis=0).T
        std  = np.std(data,axis=0).T
        ax[iN].plot(mean, label=str(layer))
        ax[iN].fill_between(range(data.shape[-1]), mean-0.5*std, mean+0.5*std,alpha=0.1)
    ax[iN].legend(title='#layers',loc=3)
    P.setp(ax[iN],ylabel='performance', xlabel='epochs', ylim=[-0.1,1.1], xlim=[-1,nEpoch-nBin+5], title=f'noise: {Noises[iN]}');
P.suptitle(f'Mean Performance of {nTrial} Trials with Optimised Learning Rates with {ExpName}', y=1.005);